### Attempt to make, new improved x-ray structure analyser

In [1]:
from typing import Tuple, Optional, List
import itertools
import pandas as pd
import numpy as np
import time
from MDAnalysis import Universe
from MDAnalysis.analysis.hydrogenbonds.hbond_analysis import HydrogenBondAnalysis as HBA
from MDAnalysis.analysis import distances

c:\Users\Rory Crean\miniconda3\envs\tools_proj_py3_11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Rory Crean\miniconda3\envs\tools_proj_py3_11\Lib\site-packages\MDAnalysis\topology\TPRParser.py:161: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  import xdrlib


In [3]:
# Time taken: 0:04:21.360000 before filtering... 
# now 3:00:00... :) 
# now with loop fixed, 2 mins! :) 

from tools_proj.contacts.contact_analysis import single_frame_contact_analysis
parm_file = r"../protein_prep/5_tleap/5HW3_BVA-1_apo.prmtop" # 
coord_file = r"../protein_prep/5_tleap/5HW3_BVA-1_apo.inpcrd"


result_no_topo = single_frame_contact_analysis(
    topology_file=parm_file,
    coordinates_file=coord_file,
    out_file="test_file.txt",
    # first_res=40,
    # last_res=75,
)
print(result_no_topo)

Sytem setup complete, identifying interactions now.
count_pre_reject=14112
count_post_reject=3429
Analysis complete, the file test_file.txt has been written to disk
Time taken: 0:02:00.938000
['ALA2 GLU5 hbond mc-mc', 'ALA2 SER6 vdw mc-mc', 'ALA3 SER6 hbond mc-mc', 'ALA3 PRO7 vdw mc-wc', 'GLU4 THR31 vdw sc-mc', 'GLU4 LEU32 vdw wc-sc', 'GLU4 TRP260 vdw sc-sc', 'GLU4 ALA264 vdw sc-sc', 'GLU5 ALA9 hbond mc-mc', 'GLU5 LEU32 vdw mc-sc', 'GLU5 GLY33 vdw sc-mc', 'GLU5 HIE34 vdw sc-wc', 'GLU5 ARG35 vdw sc-mc', 'SER6 GLU10 hbond mc-mc', 'PRO7 ILE11 hbond mc-mc', 'PRO7 GLN259 vdw sc-sc', 'LEU8 GLU12 hbond mc-mc', 'LEU8 LEU19 vdw mc-sc', 'LEU8 VAL21 hydrophobic sc-sc', 'LEU8 LEU32 vdw sc-sc', 'LEU8 GLY33 vdw sc-mc', 'LEU8 HIE34 vdw sc-wc', 'LEU8 ALA253 vdw sc-mc', 'LEU8 ALA256 hydrophobic sc-sc', 'ALA9 ARG13 hbond mc-mc', 'ALA9 HIE34 vdw wc-sc', 'GLU10 ARG13 saltbridge sc-sc', 'GLU10 ARG14 hbond mc-mc', 'ILE11 SER15 hbond mc-mc', 'ILE11 LEU19 hydrophobic sc-sc', 'ILE11 VAL249 vdw sc-wc', 'ILE11 G

In [3]:
256*256

65536

In [2]:
# quick speed test for just this module
from tools_proj.contacts.contact_analysis import _min_heavy_atom_distances
parm_file = r"../protein_prep/5_tleap/5HW3_BVA-1_apo.prmtop" # 
traj_file = r"../protein_prep/5_tleap/5HW3_BVA-1_apo.inpcrd"
universe = Universe(parm_file, traj_file) 

_min_heavy_atom_distances(universe=universe)

# could I use this to cut-out middle man and make a list of all pairs to go through instead?
# make a function called pre_filter_residues, 
# TODO, this is a good idea... spit out a list of tuples to actually test?


array([[ 0,  1,  4, ..., 15, 12, 14],
       [ 1,  0,  1, ..., 13, 10, 11],
       [ 4,  1,  0, ..., 10,  8, 10],
       ...,
       [15, 13, 10, ...,  0,  1,  3],
       [12, 10,  8, ...,  1,  0,  1],
       [14, 11, 10, ...,  3,  1,  0]])

In [ ]:
# compare to above, make sure they produce same amount of data. 
result_p1 = single_frame_contact_analysis(
    topology_file=parm_file,
    coordinates_file=coord_file,
    out_file="test_file.txt",
    first_res=1,
    last_res=125,
)

result_p2 = single_frame_contact_analysis(
    topology_file=parm_file,
    coordinates_file=coord_file,
    out_file="test_file.txt",
    first_res=126,
)


In [ ]:
len(result_p1), len(result_p2)

In [ ]:
437+308

In [ ]:
len(result_no_topo)

In [ ]:
# compare to above, make sure they produce same amount of data. 
coord_file = r"../protein_prep/5_tleap/5HW3_BVA-1_apo_postleap.pdb" # 
result_p1_pdb = single_frame_contact_analysis(
    coordinates_file=coord_file,
    out_file="test_file.txt",
    first_res=1,
    last_res=125,
)

result_p2_pdb = single_frame_contact_analysis(
    coordinates_file=coord_file,
    out_file="test_file.txt",
    first_res=126,
)


In [ ]:
len(result_p2_pdb), len(result_p1_pdb)

In [ ]:
matching = [s for s in result_no_topo if "GLY265" in s]
matching

In [ ]:
matching = [s for s in result_no_topo if "hbond" in s]
print(len(matching))

#### MDAnalysis, Hbond selection work
Can have a subroutine, that chooses if a user has a toplogy file then we go that route... 

In [ ]:
from MDAnalysis.analysis.hydrogenbonds.hbond_analysis import HydrogenBondAnalysis as HBA

parm_file = r"4YFM_MAB-1_apo.prmtop" 
traj_file = r"4YFM_MAB-1_test_traj.nc" 
universe = Universe(parm_file, traj_file) 

In [ ]:
hbonds = HBA(universe=universe)
hbonds.run()

In [ ]:

# MDAnalysis.exceptions.NoDataError will occur if topology has no charge info.

# hbond_results is a nested array.
# outer layer is a hbond found.
# inner layer is: frame, donor, hydrogen, acceptor index, distance, angle
hbond_results = hbonds.results.hbonds

# Make list of all hbond residue pairs.
hbond_pairs = []
for observation in hbond_results:
    donor_atom, acceptor_atom = observation[1], observation[3]

    donor_resid = _atom_num_to_res_info(
        atom_num=donor_atom, universe=universe)[1]
    acceptor_resid = _atom_num_to_res_info(
        atom_num=acceptor_atom, universe=universe)[1]

    if (donor_resid, acceptor_resid) not in hbond_pairs:
        hbond_pairs.append((donor_resid, acceptor_resid))

print(hbond_pairs)

In [ ]:
coord_file = r"../protein_prep/5_tleap/5HW3_BVA-1_apo_postleap.pdb"

result_no_topo = single_frame_contact_analysis(
    coordinates_file=coord_file,
    out_file="test_file.txt",
)
result_no_topo

In [ ]:
result_yes_topo = single_frame_contact_analysis(topology_file=r"WT_PTP1B_Closed.pdb",
                              coordinates_file=r"WT_PTP1B_Closed.pdb",
                             out_file="test_file.txt",
                             first_res=1,
                             last_res=298)
result_yes_topo

### Testing multiple frame analysis

In [ ]:
from tools_proj.contacts.contact_analysis import multi_frame_contact_analysis

# # Using pdb as both 
# parm_file = r"4YFM_MAB-1_test_traj.pdb" 
# traj_file = r"4YFM_MAB-1_test_traj.pdb" # 5 frames. 

# results_pdb = multi_frame_contact_analysis(topology_file=parm_file,
#                              trajectory_file=traj_file,
#                              out_file="traj_test.txt",
#                              report_time_taken = True
# )
# results_pdb

In [ ]:
# results_pdb_single = multi_frame_contact_analysis(#topology_file=parm_file,
#                              trajectory_file=traj_file,
#                              out_file="traj_test.txt",
#                              first_res=1,
#                              #last_res=282, 
#                              report_time_taken = True
# )
# results_pdb_single

In [ ]:
# Using prmtop and traj...
parm_file = r"4YFM_MAB-1_apo.prmtop" 
traj_file = r"4YFM_MAB-1_test_traj.nc" # 5 frames. 

results_prmtop_traj = multi_frame_contact_analysis(topology_file=parm_file,
                             trajectory_file=traj_file,
                             out_file="doesn't matter now",
                             last_res=100, 
                             report_time_taken = True
)
results_prmtop_traj

#### New form of pymol projections...

Draw links no longer works

In [ ]:
mean_values = results_prmtop_traj.mean().to_dict()
print(mean_values)

In [ ]:
import re
pymol_mean_values = {}

INTERACTION_TO_COLOR = {"saltbridge": "blue", "vdw": "green", "cationpi": "yellow", "pipi":"orange"}

res_res_scores, res_res_colors = {}, {}
for key, score in mean_values.items():
    res1, res2, interaction_type, residue_parts = key.split(" ")
    res1_numb = int(re.sub(r"[^0-9]", "", res1))
    res2_numb = int(re.sub(r"[^0-9]", "", res2))
    reformatted_key = (res1_numb, res2_numb)
    res_res_scores[reformatted_key] = score
    res_res_colors[reformatted_key] = INTERACTION_TO_COLOR[interaction_type]

print(res_res_colors)
print(res_res_scores)

In [ ]:
from tools_proj.pymol_projections import project_pymol_res_res_scores, project_pymol_per_res_scores, gen_per_res_scores

project_pymol_res_res_scores(res_res_scores=res_res_scores, 
                             res_res_colors=res_res_colors, 
                             out_file="test_pymol_out.py")

# no colors 
project_pymol_res_res_scores(res_res_scores=res_res_scores, 
                             out_file="test_pymol_out_no_colors.py")

In [ ]:
print(res_res_scores)

In [ ]:
per_res_scores = gen_per_res_scores(res_res_scores=res_res_scores)

In [ ]:
project_pymol_per_res_scores(per_res_scores=per_res_scores, out_file="test_pymol_per_res_out.py")

### (Side Chain) Hbonds

Can have a protocol if topology provided, but otherwise, this will work with standard 20 amino acids 
Could the expansion just be that I add the backbone names to each one, and then select from the whole aa and not from the side chain? 
Once a hydrogen bond found, can label it based on the atoms used perhaps?

Hbond defintion:
- up till 3.5 A, reducing between 2 and 3.5 good.
- 180 +- 45 degrees?, towards 180 = better.
- Could try to make a model that swaps between 0 and 1 for this? 

In [ ]:
MC_HBOND_ACCEPTORS = ["O", "OXT"]
MC_HBOND_DONORS = [{"N": ["H"]}]
 
SC_HBOND_RES_ATOMS_ACCEPTOR = {"ASN": ["OD1"], 
                                "GLN": ["OE1"], 
                                "SER": ["OG"], 
                                "THR": ["OG1"], 
                                "TYR": ["OH"], 
                                "GLU": ["OE1", "OE2"], 
                                "GLU": ["OE1", "OE2"],
                                "ASP": ["OD1", "OD2"],
                                "ASH": ["OD1", "OD2"],
                                "HIE": ["ND1"],
                                "HID": ["NE2"]
}
# List of possible combinations? 
# Hydrogens that match the pair? 
SC_HBOND_RES_ATOMS_DONOR = {"ASN": [{"ND2": ["HD21", "HD22"]}], 
                            "ARG": [{"NE": ["HE"]}, {"NH1": ["HH11", "HH12"]}, {"NH2": ["HH21", "HH22"]}], 
                            "GLN": [{"NE2": ["HE21", "HE22"]}],
                            "SER": [{"OG": ["HG"]}],
                            "THR": [{"OG1": ["HG1"]}],
                            "TYR": [{"OH": ["HH"]}], 
                            "HIE": [{"NE2": ["HE2"]}], 
                            "HID": [{"ND1": ["HD1"]}], 
                            "HIP": [{"NE2": ["HE2"]}, {"ND1": ["HD1"]}],  
                            "LYS": [{"NZ": ["HZ1", "HZ2", "HZ3"]}], 
                            "LYN": [{"NZ": ["HZ2", "HZ3"]}]
}

# cutoffs. 
HB_DIST_CUTOFF = 3.5 # Donor-acceptor Distance
HB_ANGLE_IDEAL = 180 
HB_ANGLE_TOLERANCE = 45 #

In [ ]:
parm_file = r"WT_PTP1B_Closed.pdb" 
traj_file = r"WT_PTP1B_Closed.pdb" 
universe = Universe(parm_file, traj_file) 

In [ ]:
res1, res2 = 39, 63 # 39, 63 # Hbond, 
# res1_mc_sele = "name C CA O N H and resid " + str(res1)
# res1_mc_atoms = universe.select_atoms(res1_mc_sele)
res1_name = universe.residues.resnames[res1-1] # 0-indexed
res2_name = universe.residues.resnames[res2-1] # 0-indexed
res1_name, res2_name

could make a complete list of acceptors and donors for each pair and then test each set one at a time? 

- yep, make a function, called prep_acceptor_donors 

In [ ]:
from tools_proj.contacts.utils import angle_between_two_vectors

In [ ]:
def _prep_donor_acceptors(res1_name:str, res2_name:str) -> Tuple[list, list]:
    """
    Prepare the donor and acceptors combinations. 
    """
    # start with the main chain donor and acceptors
    res1_acceptors, res2_acceptors = ["O", "OXT"], ["O", "OXT"]
    res1_donors, res2_donors = [{"N": ["H"]}], [{"N": ["H"]}]

    # add in side chain donor and acceptors. 
    if res1_name in SC_HBOND_RES_ATOMS_ACCEPTOR:
        res1_acceptors += SC_HBOND_RES_ATOMS_ACCEPTOR[res1_name]

    if res1_name in SC_HBOND_RES_ATOMS_DONOR:
        res1_donors += SC_HBOND_RES_ATOMS_DONOR[res1_name]

    if res2_name in SC_HBOND_RES_ATOMS_ACCEPTOR:
        res2_acceptors += SC_HBOND_RES_ATOMS_ACCEPTOR[res2_name]

    if res2_name in SC_HBOND_RES_ATOMS_DONOR:
        res2_donors += SC_HBOND_RES_ATOMS_DONOR[res2_name]

    # make all possible combinations. 
    res1_don_res2_acc = list(itertools.product(res1_donors, res2_acceptors))
    res2_don_res1_acc = list(itertools.product(res2_donors, res1_acceptors))
    return res1_don_res2_acc, res2_don_res1_acc

In [ ]:
def _identify_hbond(donor_acceptor_combos, donor_res_numb, acceptor_res_numb):
    """ 
    Given a combination of donor and acceptor atoms, test if any hydrogen bonds
    are present. 
    """
    hbonds_found = []
    for donor_combo, acceptor in donor_acceptor_combos:
        donor = list(donor_combo.keys())[0]
        donor_str = "name " + donor + " and resid " + str(donor_res_numb)
        donor_atom =  universe.select_atoms(donor_str)
        
        acceptor_str = "name " + acceptor + " and resid " + str(acceptor_res_numb)
        acceptor_atom = universe.select_atoms(acceptor_str)
        
        # can occur if atom names don't exist for that residue (e.g. OXT).
        if (len(donor_atom) == 0) or (len(acceptor_atom) == 0):
            continue 
        
        # remove donor_acceptor that fail distance test. 
        d_a_dist = distances.distance_array(acceptor_atom.positions, donor_atom.positions)
        if d_a_dist > HB_DIST_CUTOFF:
            continue

        for hydrogen in list(donor_combo.values())[0]:
            hydrogen_str = "name " + hydrogen + " and resid " + str(donor_res_numb)
            hydrogen_atom = universe.select_atoms(hydrogen_str)

            dh_vector = donor_atom.positions[0] - hydrogen_atom.positions[0]
            ha_vector = acceptor_atom.positions[0] - hydrogen_atom.positions[0] 
            dha_angle = angle_between_two_vectors(dh_vector, ha_vector) 

            delta_dha_ideal = min(
                np.abs(dha_angle - HB_ANGLE_IDEAL),
                np.abs(dha_angle - HB_ANGLE_IDEAL - 180)
            )
            if delta_dha_ideal < HB_ANGLE_TOLERANCE:
                hbonds_found.append( (donor_res_numb, donor, acceptor_res_numb, acceptor) )
    return hbonds_found

In [ ]:
res1_don_res2_acc, res2_don_res1_acc = _prep_donor_acceptors(res1_name, res2_name)
print(res1_don_res2_acc)
print(res2_don_res1_acc)

In [ ]:
hbonds_found = []
hbonds_found += _identify_hbond(donor_acceptor_combos=res1_don_res2_acc, 
              donor_res_numb=res1, 
               acceptor_res_numb=res2) 

hbonds_found += _identify_hbond(donor_acceptor_combos=res2_don_res1_acc, 
              donor_res_numb=res2, 
               acceptor_res_numb=res1) 

hbonds_found

In [ ]:
if len(hbonds_found) == 0:
    #return None
    print("not possible")

In [ ]:
# figure out formatting. 
# determine which parts (mainchain or sidechain or both) are involved for both residues.
# wc = whole chain, i.e. both mc and sc interacting for that residue.
hbonds_formatted = []
for hbond in hbonds_found:
    donor_res_numb, donor, acceptor_res_numb, acceptor = hbond

    if donor == "N":
        donor_part = "mc"
    else: 
        donor_part = "sc"

    if acceptor in ["O", "OXT"]:
        acceptor_part = "mc"
    else: 
        acceptor_part = "sc"

    if donor_res_numb < acceptor_res_numb:
        hbond_str = res1_name + str(res1) + " " + res2_name + str(res2) + " hbond " + donor_part + "-" + acceptor_part
    else: 
        hbond_str = res1_name + str(res1) + " " + res2_name + str(res2) + " hbond " +  acceptor_part + "-" + donor_part 

    hbonds_formatted.append(hbond_str)
hbonds_formatted

In [ ]:
# TODO - be cautious that multple hbonds could be found... 

### Metals?
